1. Read the dataset, show its head, shape and description

In [43]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import KBinsDiscretizer
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
df = pd.read_csv('data.csv', sep=';')
df.head(5)

,Date,Time,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN,NaN


In [6]:
print(f'size of df: {df.shape}\n')
print(f'brief description of df:\n{df.describe()}')
print(f'infos about df datas')
df.isnull().sum()

size of df: (9471, 18)

brief description of df:
               X00          X01          X02          X03          X04  \
count  7765.000000  8991.000000   914.000000  9357.000000  8991.000000   
mean      2.127521  1099.833166   218.811816     9.688704   939.153376   
std       1.463171   217.080037   204.459921     7.559785   266.831429   
min       0.000000   647.000000     7.000000     0.000000   383.000000   
25%       1.000000   937.000000    67.000000     4.000000   734.500000   
50%       1.800000  1063.000000   150.000000     7.900000   909.000000   
75%       2.900000  1231.000000   297.000000    13.600000  1116.000000   
max      11.900000  2040.000000  1189.000000    63.700000  2214.000000   

               X05          X06          X07          X08          X09  \
count  7718.000000  8991.000000  7715.000000  8991.000000  8991.000000   
mean    246.896735   835.493605   113.091251  1456.264598  1022.906128   
std     212.979168   256.817320    48.370108   346.206794   39

,0
Date,114
Time,114
X00,1706
X01,480
X02,8557
X03,114
X04,480
X05,1753
X06,480
X07,1756


2. Eliminate totally null columns and totally null rows, eliminate columns
with less than 1/3 of non null values; fill the remaining NaN values with
the mean of the column

In [12]:
df2 = df.drop(columns=['X02', 'X13', 'X14', 'X15'])

nanrows = df2.isnull().sum(axis=1)
print("rows missing values:")
print(nanrows.value_counts().sort_index())

rows missing values:
0     7024
1      372
2      407
3     1188
8      318
9        4
10      13
11      31
14     114
Name: count, dtype: int64


In [21]:
numeric_cols = df2.select_dtypes(include=[np.number]).columns

df2 = df2.dropna(axis=0, how='all', subset = numeric_cols)

#pandas rompe con le prime 2 colonne, vuole solo valori numerici
df2[numeric_cols] = df2[numeric_cols].fillna(df2[numeric_cols].mean())

print(f'{df2.shape}')
#controllo i nan dopo averci lavorato su
df2.isnull().sum()

(9357, 14)


,0
Date,0
Time,0
X00,0
X01,0
X03,0
X04,0
X05,0
X06,0
X07,0
X08,0


3. Drop Time, convert Date from string to datetime and group by Date
using mean as aggregate function

In [22]:
df2 = df2.drop(columns=['Time'])
df2.head(5)

,Date,X00,X01,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12
0,10/03/2004,2.6,1360.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004,2.0,1292.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004,2.2,1402.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,10/03/2004,2.2,1376.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,10/03/2004,1.6,1272.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [25]:
#per la conversione esiste la funzione to_datetime di pandas
df2['Date'] = pd.to_datetime(df2['Date'], format='%d/%m/%Y')
df2.head(5)

,Date,X00,X01,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12
0,2004-03-10,2.6,1360.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,2004-03-10,2.0,1292.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2004-03-10,2.2,1402.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2004-03-10,2.2,1376.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2004-03-10,1.6,1272.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


`groupby('Date').mean()`

esegue un'operazione di **Aggregazione**, trasformando la granularità dei dati.

Il processo segue il paradigma **"Split-Apply-Combine"**:

Ecco cosa succede passo dopo passo:

1. Raggruppamento (Split & Group)
* **Azione:** Il sistema scansiona la colonna `'Date'`.
* **Logica:** Prende tutte le righe che condividono la stessa etichetta (es. tutte le 24 registrazioni orarie del `"10/03/2004"`) e le isola in un "sacchetto" virtuale (un gruppo).

2. Calcolo (Apply Aggregation)
* **Azione:** Applica la funzione `.mean()` (media aritmetica).
* **Logica:** All'interno di quel "sacchetto", somma tutti i valori dei sensori e divide per il numero di osservazioni (24).
    * *Nota:* Questo elimina il rumore delle fluttuazioni orarie, restituendo un valore "rappresentativo" della giornata.

3. Risultato (Combine & Reduce)
* **Azione:** Crea una nuova riga unica.
* **Effetto "Schiacciamento":** Le 24 righe originali spariscono. Al loro posto, otteniamo **una singola riga** per quella data che contiene i valori medi.

In [28]:
# 3. Raggruppiamo per Data e calcoliamo la MEDIA
# Questo fonde le 24 righe orarie di ogni giorno
# in un'unica riga giornaliera
df2Day = df2.groupby('Date').mean()
print(f'dimensione con date non raggruppate: {df2.shape}')
print(f'dimensione con date raggruppate: {df2Day.shape}')

dimensione con date non raggruppate: (9357, 13)
dimensione con date raggruppate: (391, 12)


4. Preparation of the boolean matrix
* Discretise continuous values with two bins, kmeans strategy and
onehot-dense encoding
* Discretization/encoding generates 0/1 values; convert the binary
values obtained into boolean, as requested by Apriori

Data Preprocessing: Discretizzazione con `KBinsDiscretizer`

L'algoritmo **Apriori** (che utilizzeremo per l'Association Rule Mining) opera su insiemi di elementi (transazioni) e richiede dati categorici o booleani. Non è in grado di interpretare valori numerici continui come la temperatura o l'umidità.

Pertanto, è necessario trasformare le variabili continue in etichette discrete. Utilizziamo `KBinsDiscretizer` con la seguente configurazione:


1. `n_bins=2` (Discretizzazione Binaria)
Definisce il numero di intervalli ("cestini") in cui dividere i dati.
* **Obiettivo:** Vogliamo semplificare la complessità riducendo il segnale a due soli stati fondamentali: **Basso** e **Alto** (Low/High).

2. `strategy='kmeans'` (Clustering Unidimensionale)
Definisce la logica matematica per stabilire i confini (thresholds) tra i gruppi.
* **Perché non usare la media?** Un taglio semplice o uniforme potrebbe separare dati che in realtà sono simili.
* **Il metodo K-Means:** L'algoritmo analizza la distribuzione dei dati e cerca i "gruppi naturali" (cluster).
    * *Esempio:* Se abbiamo temperature {10, 12\} e {28, 30}, K-Means capisce che esistono due cluster distinti e posiziona il taglio nella zona vuota tra i due gruppi, minimizzando la varianza interna. È un approccio **adattivo** alla distribuzione reale dei dati.

3. `encode='onehot-dense'` (Binarizzazione)
Trasforma il risultato in una matrice densa di booleani (Vero/Falso), pronta per l'algoritmo Apriori.
Invece di avere una singola colonna con stringhe ("Basso", "Alto"), il sistema esplode l'informazione in colonne separate per ogni stato.

Esempio della Trasformazione

**PRIMA (Input Numerico Continuo)**
| Giorno | Temperatura |
| :--- | :--- |
| Lunedì | 10°C |
| Martedì | 30°C |

**KBinsDiscretizer**

**DOPO (Output Booleano Denso)**
| Giorno | Temp_BASSA | Temp_ALTA |
| :--- | :--- | :--- |
| Lunedì | **True** (1) | False (0) |
| Martedì | False (0) | **True** (1) |

> **Sintesi:** Stiamo trasformando una tabella numerica complessa in una "scacchiera" di interruttori logici che descrivono lo stato dei sensori, permettendo all'algoritmo di trovare regole del tipo: *"Se Temp_ALTA = True, allora CO_ALTO = True"*.

In [31]:
# 1. Configurazione del Discretizzatore
# n_bins=2: Divide ogni colonna in 2 gruppi (Basso e Alto)
# strategy='kmeans': Trova la soglia di separazione ottimale usando il clustering
# encode='onehot-dense': Crea colonne separate per ogni gruppo (es. Colonna_Basso, Colonna_Alto)

est = KBinsDiscretizer(n_bins=2, encode='onehot-dense', strategy='kmeans')
data_discretized = est.fit_transform(df2Day)

feature_names = est.get_feature_names_out(df2Day.columns)
dfB = pd.DataFrame(data_discretized, columns=feature_names, index=df2Day.index )
dfB = dfB.astype(bool)
dfB.head(5)

,X00_0.0,X00_1.0,X01_0.0,X01_1.0,X03_0.0,X03_1.0,X04_0.0,X04_1.0,X05_0.0,X05_1.0,...,X08_0.0,X08_1.0,X09_0.0,X09_1.0,X10_0.0,X10_1.0,X11_0.0,X11_1.0,X12_0.0,X12_1.0
Date,,,,,,,,,,,,,,,,,,,,,
2004-03-10,True,False,False,True,True,False,True,False,True,False,...,False,True,False,True,True,False,False,True,True,False
2004-03-11,True,False,False,True,True,False,True,False,True,False,...,False,True,True,False,True,False,False,True,True,False
2004-03-12,False,True,False,True,False,True,False,True,True,False,...,False,True,False,True,True,False,True,False,True,False
2004-03-13,False,True,False,True,False,True,False,True,True,False,...,False,True,False,True,True,False,False,True,True,False
2004-03-14,False,True,False,True,True,False,False,True,True,False,...,False,True,False,True,True,False,True,False,True,False


5. Set the names of two columns generated by the discretisation of each attribute A to A_low, A_high (with discretisation one-hot-encoding, each original column generates two columns, the first is for the low val
ues, the second for the high values)

`regex=False`

Durante la fase di pulizia dei nomi delle colonne (in particolare la rimozione del suffisso `_0.0`), abbiamo specificato esplicitamente il parametro `regex=False`. Questa scelta è dettata da una precisa necessità di sicurezza sintattica.

1. La Natura delle Regular Expressions (Regex)
Le espressioni regolari sono strumenti per il pattern matching. In questo linguaggio, alcuni caratteri hanno significati speciali (sono detti **Metacaratteri**).
* **Il caso del Punto (`.`):** Nel linguaggio Regex, il punto non è un semplice segno di punteggiatura, ma un **Jolly (Wildcard)** che significa *"Qualsiasi singolo carattere"*.

2. Il Rischio di Ambiguità
Se avessimo omesso il parametro (o impostato `regex=True`), il comando di sostituzione della stringa `_0.0` sarebbe stato interpretato come:
> *"Trova un underscore, seguito da uno zero, seguito da **qualsiasi cosa**, seguito da uno zero".*
Questo avrebbe comportato il rischio di "falsi positivi".
* *Esempio di rischio:* Se una colonna si fosse chiamata `_0A0` o `_0-0`, il sistema l'avrebbe modificata erroneamente, corrompendo i dati.

3. La Soluzione: Matching Letterale
Impostando `regex=False`, forziamo Pandas a trattare la stringa di input come una **Stringa Letterale** (Literal String).
* **Effetto:** Il punto `.` perde il suo superpotere di jolly e viene cercato esclusivamente come carattere tipografico.
* **Risultato:** L'algoritmo sostituisce la sequenza `_0.0` se e solo se trova esattamente quei caratteri, garantendo un'operazione chirurgica e priva di effetti collaterali.

In [33]:
new_columns = dfB.columns.str.replace('_0.0', '_low', regex=False)
new_columns = new_columns.str.replace('_1.0', '_high', regex=False)

# nuove etichette al dataframe
dfB.columns = new_columns
dfB.head()

,X00_low,X00_high,X01_low,X01_high,X03_low,X03_high,X04_low,X04_high,X05_low,X05_high,...,X08_low,X08_high,X09_low,X09_high,X10_low,X10_high,X11_low,X11_high,X12_low,X12_high
Date,,,,,,,,,,,,,,,,,,,,,
2004-03-10,True,False,False,True,True,False,True,False,True,False,...,False,True,False,True,True,False,False,True,True,False
2004-03-11,True,False,False,True,True,False,True,False,True,False,...,False,True,True,False,True,False,False,True,True,False
2004-03-12,False,True,False,True,False,True,False,True,True,False,...,False,True,False,True,True,False,True,False,True,False
2004-03-13,False,True,False,True,False,True,False,True,True,False,...,False,True,False,True,True,False,False,True,True,False
2004-03-14,False,True,False,True,True,False,False,True,True,False,...,False,True,False,True,True,False,True,False,True,False


6. find a value of min_support such that the Apriori algorithm generates
at least 8 frequent itemsets with at least 2 items, output the result

In [42]:
import warnings
warnings.filterwarnings('ignore')

minsupp = None
freqset = None

for i in range(90, 0, -1):
    sup = i / 100.0

    # Calcoliamo apriori
    curset = apriori(dfB, min_support=sup, use_colnames=True)

    curset['length'] = curset['itemsets'].apply(lambda x: len(x))

    valid_itemsets = curset[curset['length'] >= 2]

    if len(valid_itemsets) >= 8:
        print(f"Supporto minimo ideale: {sup}\n")
        print(f"\nNumero di itemsets trovati (con lunghezza >= 2): {len(valid_itemsets)}")

        minsupp = sup
        freqset = valid_itemsets
        break

print(f"\n itemsets frequenti trovati: {freqset}")

Supporto minimo ideale: 0.51


Numero di itemsets trovati (con lunghezza >= 2): 8

 itemsets frequenti trovati:      support                     itemsets  length
12  0.595908           (X05_low, X00_low)       2
13  0.519182           (X07_low, X00_low)       2
14  0.524297           (X09_low, X00_low)       2
15  0.621483           (X07_low, X05_low)       2
16  0.537084          (X08_high, X05_low)       2
17  0.529412           (X09_low, X05_low)       2
18  0.549872          (X08_high, X07_low)       2
19  0.514066  (X05_low, X07_low, X00_low)       3


7. find the minimum metric threshold such that at least 100 association
rules are extracted from the frequent itemsets found and show the met
rics used and the best 10 rules by descending confidence and support

Association Rule Mining & Algoritmo Apriori

L'obiettivo di questa fase è scoprire relazioni nascoste tra le variabili (nel nostro caso, i sensori) utilizzando la tecnica nota come **Market Basket Analysis**.

1. Il Concetto: il Market Basket Analysis nasce nell'ambito retail per rispondere a domande strategiche: *"Se un cliente acquista il prodotto A, con quale probabilità acquisterà anche il prodotto B?"*.

Nel nostro contesto industriale, trasliamo il concetto:
* **Cliente** $\rightarrow$ Istante Temporale (un giorno o un'ora specifica).
* **Carrello della spesa** $\rightarrow$ Stato dei Sensori in quel momento.
* **Prodotti** $\rightarrow$ Valori discretizzati dei sensori (es. `Sensor_3_High`, `Sensor_4_Low`).

L'obiettivo è trovare regole di associazione nella forma:
$$\{X\} \rightarrow \{Y\}$$
Che si legge: *"La presenza dell'insieme X implica la presenza dell'insieme Y"*.

---
2. Preparazione dei Dati: Discretizzazione e Binarizzazione

Gli algoritmi di associazione (come Apriori) non gestiscono valori numerici continui (es. 12.5°C), ma lavorano su stati booleani (Presenza/Assenza).

Il processo di trasformazione avviene in due step:
1.  **Discretizzazione (Binning):** Riduciamo la complessità numerica in categorie semantiche.
    * *Input:* 12.5, 12.7, 12.9 $\rightarrow$ *Output:* "Alto" (High).
2.  **One-Hot Encoding:** Trasformiamo le categorie in una matrice booleana. Ogni possibile stato diventa una colonna (feature) che può essere `True` (1) o `False` (0).

---
**Metriche Fondamentali**

Per valutare la bontà di una regola, utilizziamo due metriche statistiche chiave: Supporto e Confidenza.


A. Support (Frequenza)
Misura la **popolarità** di un itemset nel dataset. Ci dice quanto spesso una combinazione di eventi appare globalmente.

$$Support(X \cup Y) = \frac{\text{Freq}(X \cup Y)}{N_{tot}}$$

* *Esempio:* Se su 100 giorni, il Sensore 3 e il Sensore 4 sono entrambi "Alti" in 48 giorni:
    * $Support = 0.48$ (48%).
    * **Significato:** È un evento frequente, strutturale nel sistema.

B. Confidence (Affidabilità)
Misura la **forza dell'implicazione**. È una probabilità condizionata: dato che so che $X$ è accaduto, quante probabilità ho che accada anche $Y$?

$$Confidence(X \rightarrow Y) = \frac{Support(X \cup Y)}{Support(X)} = P(Y | X)$$

* *Esempio:* Ogni volta che il Sensore 3 è "Alto", il Sensore 4 è *sempre* "Alto".
    * $Confidence = 1.0$ (100%).
    * **Significato:** La regola è una certezza. Non è una coincidenza, è una legge deterministica del sistema.

---

**Algoritmo Apriori (Efficienza Computazionale)**

Calcolare tutte le possibili combinazioni in un dataset è computazionalmente costoso. L'algoritmo Apriori utilizza un principio intelligente (la "Apriori Property") per ridurre i calcoli:

> **Principio di Monotonicità:** Se un insieme di oggetti (itemset) è raro (infrequente), allora tutti i suoi sovra-insiemi saranno anch'essi rari.



* **Funzionamento:** Se l'algoritmo scopre che l'evento "A" è raro (sotto la soglia di supporto minimo), smette immediatamente di analizzare "A+B", "A+C", "A+B+C". Questo processo di pruning rende l'analisi più veloce.

---
Se viene chiesto di interpretare i risultati sui sensori:

1.  **Analisi:** "Abbiamo trovato la regola `{X03_High} -> {X04_High}`."
2.  **Metriche:** "La regola ha un **Supporto di 0.48** (accade quasi metà del tempo) e una **Confidenza di 1.0** (certezza assoluta)."
3.  **Conclusione:** "Questo indica una **correlazione fisica diretta**: ogni volta che il sensore 3 sale, il sensore 4 lo segue inevitabilmente. Potrebbero essere fisicamente vicini o parte dello stesso processo termodinamico."

In [49]:
from mlxtend.frequent_patterns import apriori, association_rules

# FASE 1: Generazione dei Frequent Itemsets (Teoria: Supporto)
# TEORIA: L'algoritmo Apriori trova gruppi di oggetti che appaiono spesso insieme.
# "min_support=0.1": Impostiamo il Supporto al 10%.
# GIUSTIFICAZIONE: Al punto precedente (supporto 0.51) avevamo solo 8 itemsets.
# Per la teoria, le Regole si creano combinando gli Itemsets. Da soli 8 itemsets
# è matematicamente impossibile generare 100 regole.
# Quindi, abbassiamo il supporto (rilassiamo il vincolo) per ampliare la nostra "base di partenza".
frequent_itemsets_broad = apriori(dfB, min_support=0.1, use_colnames=True)

print(f"Itemsets disponibili con supporto 0.1: {len(frequent_itemsets_broad)}")
# Risultato atteso: Ora ne avremo migliaia (es. 9000+), abbastanza per estrarre regole.


# FASE 2: Ricerca della Soglia di Confidenza
min_threshold_found = 0
best_rules = None

# TEORIA: Una regola è "X -> Y". La Confidenza è la probabilità P(Y|X).
# Vogliamo trovare la "confidenza minima" (threshold) che ci garantisca almeno 100 regole.
# Strategia: Partiamo dal massimo (99% di certezza) e scendiamo finché non raggiungiamo il target.
for conf in range(99, 1, -1):
    threshold = conf / 100.0

    # Generiamo le regole dagli itemsets trovati prima.
    # metric="confidence": Usiamo la confidenza come filtro di qualità.
    rules = association_rules(frequent_itemsets_broad, metric="confidence", min_threshold=threshold)

    # Controllo del vincolo richiesto dall'esercizio: "at least 100 association rules"
    if len(rules) >= 100:
        print(f"\nSoglia trovata, Min Confidence = {threshold}")
        print(f"Numero regole estratte: {len(rules)}")

        # Salviamo i risultati e interrompiamo il ciclo (break) perché abbiamo trovato
        # la soglia più alta possibile che soddisfa la richiesta.
        min_threshold_found = threshold
        best_rules = rules
        break


# FASE 3: Ranking e Visualizzazione (Teoria: Lift e Interessantezza)
if best_rules is not None:
    # TEORIA: Non tutte le regole sono uguali.
    # Le ordiniamo per:
    # 1. Confidence (Priorità 1): Vogliamo prima le regole più "certe" (probabilità più alta).
    # 2. Support (Priorità 2): A parità di certezza, preferiamo quelle più frequenti nel dataset.
    top_10_rules = best_rules.sort_values(by=['confidence', 'support'], ascending=[False, False]).head(10)

    print("\n Top 10 Regole (ordinate per Confidence e Support):")
    cols = ['antecedents', 'consequents', 'confidence', 'support']
    print(top_10_rules[cols])
else:
    print("Non sono state trovate 100 regole. Bisogna abbassare il supporto iniziale (min_support) in FASE 1.")

Itemsets disponibili con supporto 0.1: 9035

Soglia trovata, Min Confidence = 0.99
Numero regole estratte: 17858

 Top 10 Regole (ordinate per Confidence e Support):
                     antecedents consequents  confidence   support
1                     (X03_high)  (X04_high)         1.0  0.485934
0                      (X04_low)   (X03_low)         1.0  0.439898
2             (X04_low, X00_low)   (X03_low)         1.0  0.434783
36            (X09_low, X04_low)   (X03_low)         1.0  0.414322
51          (X08_high, X03_high)  (X04_high)         1.0  0.411765
30            (X04_low, X05_low)   (X03_low)         1.0  0.409207
122  (X09_low, X04_low, X00_low)   (X03_low)         1.0  0.409207
48           (X03_high, X06_low)  (X04_high)         1.0  0.404092
110  (X00_low, X04_low, X05_low)   (X03_low)         1.0  0.404092
425  (X09_low, X04_low, X05_low)   (X03_low)         1.0  0.396419
